In [2]:
import numpy as np
import collections
import os
import sys
import pickle
import random
import csv
import pandas as pd
from collections import defaultdict
from collections import Counter 
import time
import multiprocessing
import itertools
from multiprocessing import Pool, cpu_count
import multiprocessing as mp
from functools import partial


In [ ]:
# relation_dict = {}
# total_entity = 0
# for k, v in kg.items():
# #     print(k, v)
#     for pair in v:
#         if pair[1] not in relation_dict: relation_dict[pair[1]] = 0
#         relation_dict[pair[1]] += 1
#     total_entity += 1

# print(relation_dict)
# print('total_entity = ', total_entity, len(kg))
# for k, v in relation_dict.items():
#     print('relation = ', k, 'value = ', v/total_entity)

In [3]:
def construct_kg(kg_np):
    print('constructing knowledge graph ...')
    kg = dict()
    enti = 0
    rela = 0
    for triple in kg_np:
        head = triple[0]
        relation = triple[1]
        tail = triple[2]
        # treat the KG as an undirected graph
        if head not in kg:
            kg[head] = []
        kg[head].append((tail, relation))
        if tail not in kg:
            kg[tail] = []
        kg[tail].append((head, relation))
        enti = max(enti, head, tail)
        rela = max(rela, relation)
    return kg, enti, rela

In [21]:
def show_kg_info(dataset):
    data_path = f'../../../../../data/{dataset}/'
    kg_file = data_path + 'kg_final'
    if os.path.exists(kg_file + '.npy'):
        kg_np = np.load(kg_file + '.npy')
    else:
        kg_np = np.loadtxt(kg_file + '.txt', dtype=np.int64)

    n_entity = len(set(kg_np[:, 0]) | set(kg_np[:, 2]))
    n_relation = len(set(kg_np[:, 1]))

    rating_file = data_path + 'ratings_final'
    if os.path.exists(rating_file + '.npy'):
        ratings_final = np.load(rating_file + '.npy')
    else:
        ratings_final = np.loadtxt(rating_file + '.txt', dtype=np.int64)

    kg, enti, rela = construct_kg(kg_np)
    
    
    relation_dict = {}
    total_entity = 0
    entity_neighbor = 0
    entity_neighbor_20 = 0
    for k, v in kg.items():
    #     print(k, v)
        entity_neighbor += len(v)
        if len(v) >= 20:
            entity_neighbor_20 += 1
        for pair in v:
            if pair[1] not in relation_dict: relation_dict[pair[1]] = 0
            relation_dict[pair[1]] += 1
        total_entity += 1
    
    print(relation_dict)
    print('total_entity = ', total_entity, 'entity_neighbor = ', entity_neighbor, entity_neighbor/total_entity, 'entity_neighbor_20 = ', entity_neighbor_20/total_entity)
    for k, v in relation_dict.items():
        print('relation = ', k, 'value = ', v/total_entity)

In [22]:
dataset="last-fm_50core"
show_kg_info(dataset)

constructing knowledge graph ...
{0: 133416, 4: 64794, 1: 542058, 3: 35428, 2: 135780, 5: 6862, 6: 1030, 7: 6766, 8: 3000}
total_entity =  106389 entity_neighbor =  929134 8.7333652915245 entity_neighbor_20 =  0.024908590173796163
relation =  0 value =  1.2540394213687505
relation =  4 value =  0.6090291289513013
relation =  1 value =  5.095056819784
relation =  3 value =  0.33300435195367944
relation =  2 value =  1.2762597636973747
relation =  5 value =  0.06449914934814689
relation =  6 value =  0.009681452029815112
relation =  7 value =  0.06359680042109617
relation =  8 value =  0.028198403970335278


In [23]:
dataset="MovieLens-1M"
show_kg_info(dataset)

constructing knowledge graph ...
{0: 738790, 1: 718658, 9: 73730, 3: 108278, 7: 100994, 10: 52820, 2: 164128, 5: 164384, 8: 105886, 4: 224732, 6: 30718, 11: 872}
total_entity =  182011 entity_neighbor =  2483990 13.64747185609661 entity_neighbor_20 =  0.16961062792908121
relation =  0 value =  4.059040387668877
relation =  1 value =  3.9484316881946695
relation =  9 value =  0.405085406925955
relation =  3 value =  0.5948981105537577
relation =  7 value =  0.5548785512963502
relation =  10 value =  0.2902022405239244
relation =  2 value =  0.9017476965677899
relation =  5 value =  0.9031542049656339
relation =  8 value =  0.5817560477114021
relation =  4 value =  1.234716583063661
relation =  6 value =  0.1687700193944322
relation =  11 value =  0.004790919230156419


In [24]:
dataset="amazon-book_20core"
show_kg_info(dataset)

constructing knowledge graph ...
{0: 1331862, 1: 1332946, 3: 1331862, 4: 213328, 14: 74702, 10: 74852, 7: 213274, 6: 39412, 9: 100856, 13: 100552, 11: 27846, 2: 23400, 5: 80386, 8: 80252, 37: 470, 32: 640, 21: 724, 20: 4788, 15: 4794, 29: 724, 24: 328, 28: 328, 26: 560, 34: 560, 18: 7712, 16: 7760, 22: 5724, 17: 20364, 19: 20124, 12: 9504, 27: 324, 36: 322, 35: 700, 30: 606, 25: 602, 23: 634, 38: 270, 33: 954, 31: 446}
total_entity =  113487 entity_neighbor =  5115492 45.075576938327735 entity_neighbor_20 =  0.9586119996122904
relation =  0 value =  11.735811150175792
relation =  1 value =  11.745362905002334
relation =  3 value =  11.735811150175792
relation =  4 value =  1.8797571528016426
relation =  14 value =  0.6582427943288659
relation =  10 value =  0.6595645316203618
relation =  7 value =  1.879281327376704
relation =  6 value =  0.3472820675495872
relation =  9 value =  0.8887009084740983
relation =  13 value =  0.8860221875633333
relation =  11 value =  0.24536731079330673
r